# Project 1.) First Cloud Function

### Description : Post a cloud function that takes in a string of numbers and returns a json file that contains the the sum of all of the single digit numbers

#### Example : input ="12345"
#### output = 1+2+3+4+5 = 15
#### returns({"answer":15})

In [100]:
import json, session_info, requests
import pandas as pd 
import numpy as np

In [282]:
def num_add(request):
    try:
        request_json = request.get_json()
        numbers_string = request_json.get('numbers_string', '')

        single_digits = [int(char) for char in numbers_string if char.isdigit() and int(char) < 10]
        total_sum = sum(single_digits)
        
        return json.dumps(total_sum), 200
    
    except Exception as e:
        return json.dumps({'error': str(e)}), 400

In [283]:
url = "https://us-west2-spring2024project-424621.cloudfunctions.net/num-add"

numbers_string = input("Enter a string of numbers: ")

data = {
    "numbers_string": numbers_string
}

headers = {'Content-Type': 'application/json'}
r = requests.post(url, headers=headers, json=data)

print(f"Summation: {r.text}")

Enter a string of numbers: 112
Summation: 4


In [284]:
def sum_func_output():
    url = "https://us-west2-spring2024project-424621.cloudfunctions.net/num-add"
    
    numbers_string = input("Enter a string of numbers: ")
    
    data = {
        "numbers_string": numbers_string
    }

    headers = {'Content-Type': 'application/json'}
    r = requests.post(url, headers=headers, json=data)

    return print(f"Summation: {r.text}")

## 1.b.) Query your cloud function using requests for example input "012937", "2" and "9999999999999"

In [285]:
sum_func_output()

Enter a string of numbers: 012937
Summation: 22


In [286]:
sum_func_output()

Enter a string of numbers: 2
Summation: 2


In [288]:
sum_func_output()

Enter a string of numbers: 9999999999999
Summation: 117


# 1.c.) Add srborghese@g.ucla.edu to your cloud project via IAM.

Added as a viewer

# Project 2.) Automated Webscraping

### Description : Find a website that is scrapable with Beautiful soup that updates with some frequency. Build a cloud function to programatically scrape the useful content

In [8]:
from bs4 import BeautifulSoup
import re
import string
import google
from google.cloud import storage
import os
from io import StringIO
from datetime import datetime

In [9]:
#https://us-west2-spring2024project-424621.cloudfunctions.net/PullingReference

In [10]:
bucket_name='phils2024batting'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"E:\Downloads\Google Cloud Project\Credentials.json"
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob("webscrape/Phils_Batting_2024.csv")
blob.upload_from_filename(r"E:\Downloads\Google Cloud Project\Phils_Batting_2024.csv")

In [11]:
URL = "https://www.baseball-reference.com/teams/PHI/"
base = "https://www.baseball-reference.com"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

pattern = re.compile(r"/teams/PHI/\d{4}.shtml")

links = soup.find_all("a", href=True)

unique_links = set()

for link in links:
    href = link.get("href")
    if pattern.match(href):
        new_href = base + href
        unique_links.add(new_href)

unique_links = sorted(unique_links)

#for link in unique_links:
    #print(link)

target_link = None
for link in unique_links:
    if "2024.shtml" in link:
        target_link = link
        break

if target_link:
    target_page = requests.get(target_link)
    target_soup = BeautifulSoup(target_page.content, "html.parser")
    batting_table = target_soup.find("table", {"id": "team_batting"})

    if batting_table:
        df = pd.read_html(str(batting_table))[0]
        #print(df)
    else:
        print("Batting table not found.")
else:
    print("2024 link not found")

In [12]:
def batting_table_pull():
    URL = "https://www.baseball-reference.com/teams/PHI/"
    base = "https://www.baseball-reference.com"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    pattern = re.compile(r"/teams/PHI/\d{4}.shtml")

    links = soup.find_all("a", href=True)

    unique_links = set()

    for link in links:
        href = link.get("href")
        if pattern.match(href):
            new_href = base + href
            unique_links.add(new_href)

    unique_links = sorted(unique_links)

    target_link = None
    for link in unique_links:
        if "2024.shtml" in link:
            target_link = link
            break

    if target_link:
        target_page = requests.get(target_link)
        target_soup = BeautifulSoup(target_page.content, "html.parser")
        batting_table = target_soup.find("table", {"id": "team_batting"})

        if batting_table:
            df = pd.read_html(str(batting_table))[0]
            current_date = datetime.now().strftime("%Y-%m-%d")
            df["Date"] = current_date
        else:
            print("Batting table not found.")
    else:
        print("2024 link not found")
    return(df)

In [13]:
def download_data():
    
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"E:\Downloads\Google Cloud Project\Credentials.json"
    client = storage.Client()
    bucket_name='phils2024batting'
    blob = bucket.blob("webscrape/Phils_Batting_2024.csv")
    
    df = pd.read_csv(StringIO(blob.download_as_text()))

    return(df)

In [14]:
def post_data(request):
    old_data = download_data()

    new_data = batting_table_pull()

    df = pd.concat([old_data, new_data])

    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"E:\Downloads\Google Cloud Project\Credentials.json"
    client = storage.Client()
    bucket_name='phils2024batting'
    blob = bucket.blob("webscrape/Phils_Batting_2024.csv")

    df_string = df.to_csv()
    blob.upload_from_string(df_string)

    return({"request" : 200,
            "length_data" : len(df)})

## 2.b.) Query your stored files

In [22]:
url = 'https://us-west2-spring2024project-424621.cloudfunctions.net/Pull2'
response = requests.post(url)
if response.status_code == 200:
    print("Success:")
    print(response.json())
else:
    print(f"Failed with status code {response.status_code}:")
    print(response.text)

Success:
{'length_data': 478, 'request': 200}


## 2.c.) State how this could be useful in a business setting

It's an automated way to pull and look at differences over time without having to pay for stathead to view day by day statistics of ball players in order to view how they've been playing over stretches of time. That's why its been setup to update everyday with the date column in order to differentiate weeks, months days and so on. It could be very useful as a less costly but more time consuming alternative for smaller sports journalists, although I do imagine most large publications have their own teams with their own propietary software that can scour all types of sports related statistics. This is just one example of what they can do with a little help from the reference websites and maybe it can be a standalone service that allows individuals to easily pull certain days without having to get a stathead subscription. 

# Project 3.) 

### Description : Build some machine learning model using scikit learn and make it queriable using cloud functions

In [183]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib
from io import BytesIO 

In [184]:
birds=pd.read_csv("Eagles_10yr.csv")
birds=birds.drop(['Unnamed: 0'], axis=1)

In [185]:
X=birds[['MOT', 'OTotYd', 'DTotYd', 'TmScore','Home_Games']]
y=birds['Win']

In [186]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
clf = MLPClassifier(hidden_layer_sizes=(10,100,100,), max_iter=1000, random_state=42)
clf.fit(X_train,y_train)

MLPClassifier(hidden_layer_sizes=(10, 100, 100), max_iter=1000, random_state=42)

In [189]:
joblib.dump(clf, r"E:\Downloads\Google Cloud Project\NN.sav")
joblib.dump(scaler, r"E:\Downloads\Google Cloud Project\scaler.sav")

['E:\\Downloads\\Google Cloud Project\\scaler.sav']

In [191]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"E:\Downloads\Google Cloud Project\Credentials.json"
client = storage.Client()

bucket_name = "eaglesdataml"
bucket = client.get_bucket(bucket_name)
blob = bucket.blob("eagles/NN.sav")
blob.upload_from_filename(r"E:\Downloads\Google Cloud Project\NN.sav")

In [192]:
bucket_name = "eaglesdataml"
bucket = client.get_bucket(bucket_name)
blob = bucket.blob("eagles/scaler.sav")
blob.upload_from_filename(r"E:\Downloads\Google Cloud Project\scaler.sav")

In [263]:
def load_scikit_model(file_name):
    bucket_name = "eaglesdataml"
    source_blob = "eagles/" + file_name
    
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"E:\Downloads\Google Cloud Project\Credentials.json"
    client = storage.Client()
    
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob)
    
    model_data = blob.download_as_bytes()
    
    model = joblib.load(BytesIO(model_data))
    return(model)

In [264]:
model = load_scikit_model("NN.sav")
preproc = load_scikit_model("scaler.sav")

In [265]:
import warnings
def eagles_win(request_dictionary):
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            model = load_scikit_model("NN.sav")
            preproc = load_scikit_model("scaler.sav")
    
            dictionary = request_dictionary
            for key, value in dictionary.items():
                globals()[key] = value
    
            X = preproc.transform([[MOT, OTotYd, DTotYd, TmScore,Home_Games]])
    
            predictions = model.predict(X)[0]

            probability = str(round(model.predict_proba(X)[0][1]*100,2))+"%"
        
            return(probability)
    except:
        return({"status": "error"})

In [ ]:
import warnings
import google
import joblib
import pandas as pd
import requests
import sklearn
from urllib.parse import parse_qs
from google.cloud import storage
import os
from io import BytesIO

def load_scikit_model(file_name):
    bucket_name = "eaglesdataml"
    source_blob = "eagles/" + file_name

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Credentials.json"
    client = storage.Client()
    print("Client Created")
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob)

    model_data = blob.download_as_bytes()
    model = joblib.load(BytesIO(model_data))
    return model

def eagles_win(request):
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            model = load_scikit_model("NN.sav")
            preproc = load_scikit_model("scaler.sav")
            print("Models Loaded properly")
            
            q_str = request.get_data().decode()
            dictionary = {k: v[0] for k, v in parse_qs(q_str).items()}
            
            for key, value in dictionary.items():
                globals()[key] = value
            print("Variables Set")
            
            X = preproc.transform([[MOT, OTotYd, DTotYd, TmScore, Home_Games]])
            print("Hello, please work")
            predictions = model.predict(X)[0]

            probability = round(model.predict_proba(X)[0][1]*100, 2)
            print("Probabilities Calculated")
            print(predictions)
            print(probability)
            return {
                "prediction": int(predictions),
                "status": 200, 
                "prob_of_win": f"{probability}%"
            }
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return {"status": "error", "error_message": str(e)}

In [267]:
eagles_win({ 'MOT' : -3,
                      'OTotYd' : 348,
                      'DTotYd' : 244,
                      'TmScore': 20, 
                      'Home_Games':0
})

'10.95%'

In [335]:
r = requests.post('https://us-west2-spring2024project-424621.cloudfunctions.net/EaglesMachLearn',
                 {"MOT" : -3,
                      "OTotYd" : 155,
                      "DTotYd" : 175,
                      "TmScore": 20, 
                      "Home_Games":1
}) 


In [336]:
r.json()

{'prediction': 0, 'prob_of_win': '1.45%', 'status': 200}

## 3.b.) Make a user-friendly input page that takes the inputs to your ML model via widgets and displays the output. Upload a seperate .ipynb that makes this easy to use. (Next Assignment you will have tot urn this into a shareable webpage).

In [277]:
import ipywidgets as widgets
from IPython.display import display

In [278]:
r = requests.post('https://us-west2-spring2024project-424621.cloudfunctions.net/EaglesMachLearn',
                 {"MOT" : -3,
                      "OTotYd" : 348,
                      "DTotYd" : 244,
                      "TmScore": 20, 
                      "Home_Games":0
}) 

In [337]:
text_MOT = widgets.Text(
    value = "",
    placeholder = "Turnover Margin?",
    description = "Margin of Turnover",
    disabled = False
)

text_Otot = widgets.Text(
    value = "",
    placeholder = "Total Yards Gained?",
    description = "Total Yards Gained",
    disabled = False
)

text_Dtot = widgets.Text(
    value = "",
    placeholder = "Total Yards Allowed?",
    description = "Total Yards Allowed",
    disabled = False
)

text_Tms = widgets.Text(
    value = "",
    placeholder = "Points Scored?",
    description = "Points Scored",
    disabled = False
)

text_Home = widgets.Text(
    value = "",
    placeholder = "Home Game? 1 for Yes, 0 for No",
    description = "Home Game Identifier",
    disabled = False
)

button = widgets.Button(description = "Enter")

def eagles_win_prediction(button):
 r = requests.post('https://us-west2-spring2024project-424621.cloudfunctions.net/EaglesMachLearn',
                 {"MOT" : text_MOT.value,
                      "OTotYd" : text_Otot.value,
                      "DTotYd" : text_Dtot.value,
                      "TmScore": text_Tms.value, 
                      "Home_Games":text_Home.value})  
 print(r.json())


button.on_click(eagles_win_prediction)

In [334]:
display(text_MOT)
display(text_Otot)
display(text_Dtot)
display(text_Tms)
display(text_Home)
display(button)

Text(value='', description='Margin of Turnover', placeholder='Turnover Margin?')

Text(value='', description='Total Yards Gained', placeholder='Total Yards Gained?')

Text(value='', description='Total Yards Allowed', placeholder='Total Yards Allowed?')

Text(value='', description='Points Scored', placeholder='Points Scored?')

Text(value='', description='Home Game Identifier', placeholder='Home Game? 1 for Yes, 0 for No')

Button(description='Enter', style=ButtonStyle())

{'prediction': 1, 'prob_of_win': '100.0%', 'status': 200}
{'prediction': 1, 'prob_of_win': '99.19%', 'status': 200}
{'prediction': 1, 'prob_of_win': '100.0%', 'status': 200}
{'prediction': 1, 'prob_of_win': '99.86%', 'status': 200}
{'prediction': 1, 'prob_of_win': '99.86%', 'status': 200}
{'prediction': 0, 'prob_of_win': '1.45%', 'status': 200}


## 3.c.) Think of a company that would use the ML app you just built. What employees could use this app what would they use it for? Write a short paragraph.

Maybe ESPN wants a quick way to output some form of predictive data, or maybe whatever broadcast is showing a prediction of who will win as the game goes on automates the algorithm to run every minute or so (after a play). Now they can show predictive analytics of who will win the game while the game is running. This has multiple effects and can be used in a variety of ways, whether they aid the quality of the broadcast and the commentators or something a little more insidious in betting and half-time parlays and so on. This could mainly be used by individuals who work in sports analytics and visualization or even betting sites. It really depends, and it can be used a wide host of subjects